In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [2]:
# read JSON file
with open('table17aug21.json', 'r') as json_file:
    json_data = json.load(json_file)
with open('actions.json', 'r') as action_history:
    actions = json.load(action_history)

In [3]:
# get list of buckets(lists of cards) to DataFrame
buckets = pd.DataFrame(json_data['lists']).loc[:, ['id', 'name']].set_index('id')
buckets.head()

,name
id,
5e341a4871eef21fe7cd1984,Paper accepted
5ef47d2bcc3c19064902e85d,Author submitted
5ffc260ff8ef244d7cc4e865,At team
5e341a4b43b62f44e56427e2,At editor
5e341a4f1402ce58aeabda74,At author


In [4]:
# cards to DataFrame
columns = ['id', 'idList', 'closed', 'name', 'labels']
cards = pd.DataFrame(json_data['cards']).loc[:, columns].set_index('id')
cards_num = len(cards['labels'].values)     # get total number of cards to iterate

# extract labels from dictionary
card_labels = []
for i in range(cards_num):
    try:
        card_labels.append([cards['labels'].values[i][j]['name'] for j in range(len(cards['labels'].values[i]))])
    except IndexError:
        card_labels.append("")
        
#cards['labels'] = [ cards['labels'].values[i][0]['name'] for i in range(cards_num)]
cards['labels'] = card_labels
# add buckets (lists of cards) names to DataFrame 'cards'
cards = pd.merge(cards, buckets, how='left', left_on='idList', right_on='id', 
                 suffixes=('_card', '_list'), right_index=True).drop(['idList'], axis=1)

cards.head()

,closed,name_card,labels,name_list
id,,,,
5ed4ac37e2c0ae3fdd0564d4,False,Template,[],Paper accepted
5ede5220637a767c05f9edcd,True,Paper,[],Paper accepted
5ef8d7c4dba8de08b543c6bc,True,Re: RESTUD: MS 24476-3 Accepted,[],Paper accepted
5ef8d8189e41fd6f7132b08a,True,Re: RESTUD: MS 26198-3 Accepted,[],Paper accepted
5ef8d829b9b4db1d68a3660c,True,Re: RESTUD: MS 26541-2 Accepted,[],Paper accepted


In [5]:
# prepare DataFrame with actions (activities)
columns = ['idMemberCreator','data','type','date']

# filter DataFrame actions and sort in chronological order
actions_df = pd.DataFrame(actions)
action_df = actions_df.sort_values(by='date').reset_index(drop=True)

# prepare DataFrame with memebers (users)
members = pd.DataFrame(json_data['members']).loc[:, ['id', 'username']].set_index('id')
# add usernames to DataFrame 'actions'
actions_df = pd.merge(actions_df, members, how='left',
                   left_on='idMemberCreator', right_on='id',
                   suffixes=('_actions', '_members')).drop(['idMemberCreator'], axis=1)

# add actions information to DataFrame 'actions'

# prepare new columns
actions_df['card_id'] = ''
actions_df['list_before'] = ''
actions_df['list_after'] = ''

In [6]:
# add info to DataFrame
for row in range(len(actions_df)):
    actions_df.loc[row, 'card_id'] = actions_df.loc[row, 'data']['card']['id']
    try:
        actions_df.loc[row, 'list_before'] = actions_df.loc[row, 'data']['listBefore']['name']
        actions_df.loc[row, 'list_after'] = actions_df.loc[row, 'data']['listAfter']['name']
    except KeyError:
        actions_df.loc[row, 'list_before'] = ''
        actions_df.loc[row, 'list_after'] = ''

In [7]:
# drop duplicates by action id, date, action type, list after and before action
actions_df = actions_df.drop_duplicates(subset=['id','date','type','list_after','list_before']).reset_index()

In [8]:
# create table for export
cards_list = []
for row in range(len(cards)):
    # prepare empty dictionary
    card = {}
    
    # add name of card (task)
    card['name'] = cards.iloc[row,1]
    
    # add status
    card['status'] = 'Open'
    if cards.iloc[row, 0] == True:
        card['status'] = 'Closed'
    
    # add name of list (bucket)
    card['list'] = cards.iloc[row, 3]
    
    # add label
    card['label'] = cards.iloc[row, 2]
    
    # add actions to card
    # get card id
    card_id = cards.index[row]
    
    # iterate over list of action to find action for this card
    counter = 0
    for action_id in range(len(actions_df)):
        if actions_df['card_id'][action_id] == card_id:
            card['activity' + str(counter)] = actions_df['type'][action_id]
            card['date' + str(counter)] = actions_df['date'][action_id][:16]
            card['list_before' + str(counter)] = actions_df['list_before'][action_id]
            card['list_after' + str(counter)] = actions_df['list_after'][action_id]
            counter += 1
    
    # add card dict to list of cards
    cards_list.append(card)

In [9]:
# prepare new DataFrame from list of dicts
df = pd.DataFrame(cards_list).set_index('name')
# export to CSV
df.transpose().to_csv('table31aug21.csv')